In [144]:
import json
import requests
import numpy as np
import os
from dotenv import load_dotenv
import openai


load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')


def openai_complete(prompt, 
                    stops=None, 
                    max_tokens=100, 
                    temperature=0.9, 
                    engine='davinci',
                    max_completions=1):
    finished = False
    completion = ''
    n_completions = 0
    
    while not finished:
        response = openai.Completion.create(
            engine=engine, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature,
            stop=stops)
        
        n_completions += 1
        text = response.choices[0].text
        completion += text
        print(n_completions, completion)
        prompt += text
        finished = response.choices[0].finish_reason == 'stop' \
            or n_completions >= max_completions
    
    return completion.strip()


def openai_search(documents, query, engine='davinci'):
    data = json.dumps({"documents": documents, "query": query})
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer %s' % os.getenv('OPENAI_API_KEY'),
    }
    response = requests.post('https://api.openai.com/v1/engines/davinci/search', headers=headers, data=data)
    result = json.loads(response.text)
    return result


In [145]:


candidates = ['happy', 'sad', 'funny', 'crazy', 'weird', 'disgusting', 'bicycle', 'mountains', 'nature', 'robot']
query = 'Have you considered that you may feel like a robot because you are a robot?'
result = openai_search(candidates, query, engine='curie')
print(result)

scores = [doc['score'] for doc in result['data']]
ranked_queries = list(reversed(np.argsort(scores)))
options_search = [{'candidate': candidates[idx], 'score': scores[idx]} for idx in ranked_queries]
for result in options_search[:]:
    print(" -> %s : %0.2f" % (result['candidate'], result['score']))
    


{'object': 'list', 'data': [{'object': 'search_result', 'document': 0, 'score': -10.01}, {'object': 'search_result', 'document': 1, 'score': -5.976}, {'object': 'search_result', 'document': 2, 'score': -7.003}, {'object': 'search_result', 'document': 3, 'score': -1.018}, {'object': 'search_result', 'document': 4, 'score': 5.434}, {'object': 'search_result', 'document': 5, 'score': -10.251}, {'object': 'search_result', 'document': 6, 'score': -22.648}, {'object': 'search_result', 'document': 7, 'score': -21.679}, {'object': 'search_result', 'document': 8, 'score': -12.835}, {'object': 'search_result', 'document': 9, 'score': 27.077}], 'model': 'davinci:2020-05-03'}
 -> robot : 27.08
 -> weird : 5.43
 -> crazy : -1.02
 -> sad : -5.98
 -> funny : -7.00
 -> happy : -10.01
 -> disgusting : -10.25
 -> nature : -12.84
 -> mountains : -21.68
 -> bicycle : -22.65


## Emoji reactions

In [147]:
from emojis import *
import random
import numpy as np

candidates = list(emoji_docs.keys())
query = 'this is a bike'
result = openai_search(candidates, query, engine='curie')

scores = [doc['score'] for doc in result['data']]
ranked_queries = list(reversed(np.argsort(scores)))

options = [{'candidate': candidates[idx], 'score': scores[idx]} 
           for idx in ranked_queries
           if scores[idx] > 10][:4]


print(options)

option = random.choices([o['candidate'] for o in options], 
                        weights=[o['score'] for o in options], k=1)[0]
react = random.choice(emoji_docs[option]).strip()
print(option)
print(emoji_docs[option])
print(react)

# for result in options_search[:]:
#     emoji = emoji_docs[result['candidate']] #random.choice(emoji_docs[result['candidate']])
#     print(" -> %s : %0.2f %s" % (result['candidate'], result['score'], emoji))
    


[{'candidate': 'bicycle, biking', 'score': 191.402}, {'candidate': 'running, exercise', 'score': 69.08}, {'candidate': 'cool, hip', 'score': 67.19}, {'candidate': 'work, tools, building', 'score': 65.606}]
work, tools, building
['🔨', '🛠️']
🛠️


In [148]:

documents = [
    "physics",
    "history",
    "sports",
    "science",
    "philosophy",
    "social life"
]

query = 'What is the Newton\'s law of gravitation?'
query = 'When did the Ottoman empire exist?'
#query = 'How do you play cricket?'

result = openai_search(documents, query, engine='ada')

print('query: %s' % query)
#sorted(result['data'], key=lambda d:-d['score'])

for d, doc in enumerate(result['data']):
    print(" -> %s : %0.2f" % (documents[d], doc['score']))


query: When did the Ottoman empire exist?
 -> physics : -20.09
 -> history : 42.87
 -> sports : -18.17
 -> science : -25.04
 -> philosophy : -39.68
 -> social life : -31.21


In [2]:
%load_ext autoreload
%autoreload 2

from bots2 import bots
bots['sunrisesunset']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'token_env': 'DISCORD_TOKEN_SUNRISESUNSET',
 'debug': False,
 'programs': {'spotify': {'name': 'Sunrise Sunset'},
  'gpt3_prompt': [{'prompt': 'Here is a short poem about the beginning of a new day.\n\n    I\'ll tell you how the sun rose, —\n    A ribbon at a time.\n    The steeples swam in amethyst,\n    The news like squirrels ran.\n\n    The hills untied their bonnets,\n    The bobolinks begun.\n    Then I said softly to myself,\n    "That must have been the sun!" \n\n\n\n    Here is a short poem about the beginning of a new day.\n\n    Dawn\n    Is beautiful.\n    Its new and raw.\n    It\'s  beautifully honest.\n    There\'s something redeeming\n    about the early minutes our day\n    It imitates the early minutes of our existence\n    And erodes the nonsense and lies\n    Of day-to-day survival.\n\n\n\n    Here is a short poem about the beginning of a new day.\n\n    Long before the postman comes\n    The sun begins to rise,\n    Far in the East if you should look\n    You\'d f

In [4]:
from bots import bots
for s in bots.keys():
    print(s)

sunrisesunset
mesa
mechanicalduck
chatsubo
wall-e
eve
facts
philosophy
deeplearning
kitchen
qa


In [1]:
import bots2

GO!
<module 'bots2.channels' from '/home/gene/mars/discord-bots/bots2/channels.py'>
